In [28]:
import os

import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [19]:
os.chdir('D:/data_science/recommendation_ALS')
data = pd.read_csv('lastfm.csv')
data_items = data.drop(['user'],axis=1)
data_items.columns

Index(['a perfect circle', 'abba', 'ac/dc', 'adam green', 'aerosmith', 'afi',
       'air', 'alanis morissette', 'alexisonfire', 'alicia keys',
       ...
       'timbaland', 'tom waits', 'tool', 'tori amos', 'travis', 'trivium',
       'u2', 'underoath', 'volbeat', 'yann tiersen'],
      dtype='object', length=285)

In [20]:
#Normalization of data
user_magnitude = np.sqrt(np.square(data_items)).sum(axis = 1)
data_items = data_items.divide(user_magnitude, axis=0)
data_items.head()

,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.041667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
#Cosine similarity fn
def cosine_sim(data):
    sparse_mat = sparse.csr_matrix(data)
    csr=cosine_similarity(sparse_mat.transpose())
    csr = pd.DataFrame(csr, index=data.columns, columns=data.columns)
    return csr

In [50]:
csr=cosine_sim(data_items)
csr.shape

(285, 285)

In [58]:
#11 similar artist of adam green
print(csr['adam green'].sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[0:11])

adam green        1.000000
the strokes       0.201541
arcade fire       0.189482
amy winehouse     0.164834
mgmt              0.162026
the libertines    0.150996
the smiths        0.147497
air               0.142503
cat power         0.138008
babyshambles      0.093308
travis            0.091858
Name: adam green, dtype: float64


In [96]:
#top reccomadation for user 5985
index = data.ix[data.user == 5985].index.tolist()[0]
known_rating = data_items.iloc[index]
known_rating = known_rating[known_rating>0].index.tolist()
user_vector = data_items.ix[index]

C:\Users\Sourish\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\Sourish\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [102]:
user_vector.sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[0:10]

the cure           0.5
bob dylan          0.5
franz ferdinand    0.0
frank sinatra      0.0
foo fighters       0.0
flyleaf            0.0
flogging molly     0.0
finntroll          0.0
feist              0.0
deichkind          0.0
Name: 358, dtype: float64

In [103]:
#score for user 543
score = csr.dot(user_vector).div(csr.sum(axis=1))
score=score.drop(known_rating)
score.shape

(283,)

In [104]:
print(score.sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[0:10])

misfits                0.026892
yann tiersen           0.022047
the rolling stones     0.019868
the smiths             0.019472
joy division           0.018722
ramones                0.017025
johnny cash            0.015238
belle and sebastian    0.014390
tom waits              0.013955
eric clapton           0.012551
dtype: float64


In [133]:
#Neighbourhood method- only focus on 10 similar nighbour of eache item
neighbour_data = pd.DataFrame(index=csr.columns,columns=range(1,11))
for i in range(csr.shape[1]):
    neighbour_data.iloc[i,:] = csr.iloc[i].sort_values(axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')[1:11].index
neighbour_data.head()    

,1,2,3,4,5,6,7,8,9,10
a perfect circle,dredg,tool,deftones,porcupine tree,opeth,godsmack,editors,massive attack,system of a down,tenacious d
abba,michael jackson,elvis presley,robbie williams,the ting tings,frank sinatra,queen,the beatles,duffy,james blunt,madonna
ac/dc,opeth,rage against the machine,judas priest,dream theater,deep purple,nightwish,kreator,aerosmith,peter fox,metallica
adam green,the strokes,arcade fire,amy winehouse,mgmt,the libertines,the smiths,air,cat power,babyshambles,travis
aerosmith,pink,metallica,dire straits,jack johnson,judas priest,lenny kravitz,oasis,ac/dc,u2,linkin park


In [146]:
similar_list = neighbour_data.loc[known_rating].values.tolist()

In [156]:
similar_list = list(set(item for sublist in similar_list for item in sublist))

In [173]:
neighbour = csr[similar_list].loc[similar_list]
user_vector = data_items.iloc[index].loc[similar_list]
score = neighbour.dot(user_vector).div(neighbour.sum(axis=1))
score = score.drop(known_rating)
print(score.sort_values(axis=0,ascending=False,kind='quicksort', na_position='last')[0:10])

jimi hendrix           0.021535
johnny cash            0.055688
the rolling stones     0.057601
led zeppelin           0.027008
misfits                0.062199
elliott smith          0.032531
deep purple            0.021141
belle and sebastian    0.050243
ramones                0.062286
eric clapton           0.033045
depeche mode           0.020118
pink floyd             0.029004
the smiths             0.063273
joy division           0.043810
the doors              0.022966
tom waits              0.042341
yann tiersen           0.064849
dtype: float64

In [176]:
print(score.sort_values(axis=0,ascending=False,kind='quicksort', na_position='last')[0:10])
print(score.nlargest(10))

yann tiersen           0.064849
the smiths             0.063273
ramones                0.062286
misfits                0.062199
the rolling stones     0.057601
johnny cash            0.055688
belle and sebastian    0.050243
joy division           0.043810
tom waits              0.042341
eric clapton           0.033045
dtype: float64
yann tiersen           0.064849
the smiths             0.063273
ramones                0.062286
misfits                0.062199
the rolling stones     0.057601
johnny cash            0.055688
belle and sebastian    0.050243
joy division           0.043810
tom waits              0.042341
eric clapton           0.033045
dtype: float64


In [180]:
#ALS method collaborative filtering
#import implicit
als_data = pd.read_table('lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv')

In [186]:
import implicit
als_data = als_data.drop(als_data.columns[1], axis=1)
als_data.columns = ['user','artist','item']
als_data.head()

,user,artist,item
0,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
1,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
2,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
3,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
4,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691


In [198]:
als_data['item'] = als_data['item'].astype('int')

In [199]:
#Data preprocess
als_data = als_data.dropna()
als_data.shape
als_data['user'] = als_data['user'].astype('category')
als_data['artist'] = als_data['artist'].astype('category') 
als_data['user_id'] = als_data['user'].cat.codes
als_data['artist_id'] = als_data['artist'].cat.codes

In [201]:
#sparse matrix prep- user-item for recommendation and item-user for model building
sparse_item_user = sparse.csr_matrix((als_data['item'].astype(float), (als_data['artist_id'], als_data['user_id'])))
sparse_user_item = sparse.csr_matrix((als_data['item'].astype(float), (als_data['user_id'], als_data['artist_id'])))

In [207]:
mod = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

In [211]:
alpha = 15
conf = (sparse_item_user * alpha).astype('double')
mod.fit(conf,show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [01:02<00:00,  3.24s/it]


In [221]:
# Find the 10 most similar to Jay-Z
item_id = 147068 #Jay-Z
n_similar = 10
user_vec = mod.user_factors
item_vec = mod.item_factors

In [231]:
item_norm = np.sqrt(np.square(item_vec)).sum(axis = 1)
score = item_vec.dot(item_vec[item_id])/item_norm

(292363,)

In [243]:
#similar items
similar = mod.similar_items(N=10,itemid=item_id)
for item in similar:
    idx,score = item
    print(als_data.artist.loc[als_data.artist_id == idx].iloc[0])

jay-z
snoop dogg
outkast
2pac
lupe fiasco
50 cent
the roots
the streets
cypress hill
atmosphere


In [247]:
#Recommendation for user
user_id = 2025
reco = mod.recommend(user_items=sparse_user_item,userid=user_id,N=10,filter_already_liked_items=True)

In [253]:
artist = []
scores = []

for item in reco:
    idx, score = item
    artist.append(als_data.artist.loc[als_data.artist_id == idx].iloc[0])
    scores.append(score)

recommended=pd.DataFrame({'artist':artist, 'scores':scores})
recommended.sort_values(ascending=False,axis=0,by='scores',kind='quicksort',na_position='last',inplace=False)
recommended

,artist,scores
0,sly & the family stone,1.165822
1,brian wilson,1.148287
2,thelonious monk,1.126216
3,curtis mayfield,1.125114
4,funkadelic,1.122609
5,duke ellington,1.120699
6,sam cooke,1.117824
7,robert johnson,1.116980
8,neil young & crazy horse,1.113636
9,drive-by truckers,1.110915


In [251]:
(als_data.artist.loc[als_data.artist_id == idx].iloc[0])

'drive-by truckers'